In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import random
import math
from scipy.stats import norm
from scipy.optimize import minimize
import time
import ray
ray.init(num_cpus= 10, ignore_reinit_error=True)
from scipy.stats import norm
from numpy.linalg import inv
import matplotlib.pyplot as plt
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

2023-02-23 22:20:29,061	INFO worker.py:1538 -- Started a local Ray instance.


In [2]:
from DGP_functions import *
from MLE_functions import *
from SMM_functions import *
from MIEQ_functions import *
from moment_conditions import *

In [3]:
# define true parameters
alpha = 1
beta = 1
delta = 1 
mu = 2
sigma = 1 
market_number = 250

# Q1. Data Generate

In [8]:
master_data = pd.read_csv("data_observable.csv") # all good
#master_data = pd.read_csv("master_data0223.csv")  # MLE good
master_data.head(5)

,Unnamed: 0,market_id,potential_entrant,X_m,Z_fm,N_star,entrant,firm_id
0,0,0,3,4.764052,0.400157,1,False,1.0
1,1,0,3,4.764052,0.978738,1,True,2.0
2,2,0,3,4.764052,2.240893,1,False,3.0
3,3,1,2,4.624345,-0.611756,2,True,1.0
4,4,1,2,4.624345,-0.528172,2,True,2.0


In [9]:
data_observable = master_data[['market_id', 'potential_entrant', 'X_m', 'Z_fm', 'N_star', 'entrant']]
data_observable['firm_id'] = data_observable.groupby('market_id')['Z_fm'].rank(method='min', ascending = True)
data_observable.head(5)

,market_id,potential_entrant,X_m,Z_fm,N_star,entrant,firm_id
0,0,3,4.764052,0.400157,1,False,1.0
1,0,3,4.764052,0.978738,1,True,2.0
2,0,3,4.764052,2.240893,1,False,3.0
3,1,2,4.624345,-0.611756,2,True,1.0
4,1,2,4.624345,-0.528172,2,True,2.0


# Q2. MLE

In [10]:
def log_likelihood(theta, data_observable):
    delta, mu, sigma = theta
    log_sum = 0
    for i in range(market_number):
        work_data = data_observable.loc[data_observable['market_id']==i]
        potential_entrant = list(work_data['potential_entrant'])[0]
        
        if potential_entrant == 2:
            prob = prob_ent2(work_data, delta, mu, sigma)
        elif potential_entrant == 3:
            prob = prob_ent3(work_data, delta, mu, sigma)
        elif potential_entrant == 4:
            prob = prob_ent4(work_data, delta, mu, sigma)
        
        log_sum += math.log(prob)
    
    return -log_sum

In [11]:
# run the optimization
MLE_result = minimize(log_likelihood, [0.5, 1, 0.5], args = (data_observable), method='L-BFGS-B', options={'maxiter':200})

In [12]:
# store the MLE estimates
delta_hat = MLE_result.x[0]
mu_hat = MLE_result.x[1]
sigma_hat = MLE_result.x[2]

# calculate the std error using hessian information
sample_SE_delta = np.sqrt(MLE_result.hess_inv.todense()[0][0])/np.sqrt(market_number)
sample_SE_mu = np.sqrt(MLE_result.hess_inv.todense()[1][1])/np.sqrt(market_number)
sample_SE_sigma = np.sqrt(MLE_result.hess_inv.todense()[2][2])/np.sqrt(market_number)

# store the confidence level
CI_constant = norm.ppf(0.975, 0, 1)

print("MLE delta_hat: ", delta_hat)
print("MLE mu_hat: ", mu_hat)
print("MLE sigma_hat: ", sigma_hat)

print("MLE CI for delta_hat: [", delta_hat - CI_constant*sample_SE_delta,",", delta_hat + CI_constant*sample_SE_delta, "]")
print("MLE CI for mu_hat: [", mu_hat - CI_constant*sample_SE_mu,",", mu_hat + CI_constant*sample_SE_mu, "]")
print("MLE CI for sigma_hat: [", sigma_hat - CI_constant*sample_SE_sigma,",", sigma_hat + CI_constant*sample_SE_sigma, "]")

MLE delta_hat:  0.9106269397820578
MLE mu_hat:  1.9935403718653495
MLE sigma_hat:  0.9351659683064772
MLE CI for delta_hat: [ 0.839824968880798 , 0.9814289106833175 ]
MLE CI for mu_hat: [ 1.953370409613819 , 2.03371033411688 ]
MLE CI for sigma_hat: [ 0.8865395468827282 , 0.9837923897302263 ]


# Q3. SMM using market & firm level information

## (1) Correctly Specified Model

In [13]:
# do the first step estimation using W = I
SMM_result = minimize(SMM_estimator, [0.5, 1, 0.5], args = (data_observable), method='Nelder-Mead', options={'maxiter':300})

# store the weighting matrix
W_hat = SMM_weighting_matrix(SMM_result.x, data_observable)

# two step SMM
efficient_SMM_result = minimize(efficient_SMM_estimator, [0.5, 1, 0.5], args = (data_observable, W_hat), method='Nelder-Mead', options={'maxiter':300})

In [15]:
# store the efficient SMM estimates
delta_hat = efficient_SMM_result.x[0]
mu_hat = efficient_SMM_result.x[1]
sigma_hat = efficient_SMM_result.x[2]

# calculate the std error following Pakes and Pollard (1989)
sample_SE_delta, sample_SE_mu, sample_SE_sigma = Pakes_Pollard_SE(efficient_SMM_result, W_hat, 0.0005, data_observable)

# store the confidence level
CI_constant = norm.ppf(0.975, 0, 1)

print("SMM delta_hat: ", delta_hat)
print("SMM mu_hat: ", mu_hat)
print("SMM sigma_hat: ", sigma_hat)

print("SMM CI for delta_hat: [", delta_hat - CI_constant*sample_SE_delta,",", delta_hat + CI_constant*sample_SE_delta, "]")
print("SMM CI for mu_hat: [", mu_hat - CI_constant*sample_SE_mu,",", mu_hat + CI_constant*sample_SE_mu, "]")
print("SMM CI for sigma_hat: [", sigma_hat - CI_constant*sample_SE_sigma,",", sigma_hat + CI_constant*sample_SE_sigma, "]")

SMM delta_hat:  0.8914265756285141
SMM mu_hat:  1.9801378017976827
SMM sigma_hat:  1.1384379768229902
SMM CI for delta_hat: [ 0.5671497267626208 , 1.2157034244944076 ]
SMM CI for mu_hat: [ 1.6718552885558289 , 2.2884203150395366 ]
SMM CI for sigma_hat: [ 1.0094694132201447 , 1.2674065404258357 ]


## (2) Misspecified Model

In [19]:
# do the first step estimation using W = I
SMM_result2 = minimize(SMM_estimator_wrong, [0.5, 1, 0.5], args = (data_observable), method='Nelder-Mead', options={'maxiter':300})

# store the weighting matrix
W_hat2 = SMM_weighting_matrix_wrong(SMM_result2.x, data_observable)

# two step SMM
efficient_SMM_result2 = minimize(efficient_SMM_estimator_wrong, [0.5, 1, 0.5], args = (data_observable, W_hat2), method='Nelder-Mead', options={'maxiter':300})

In [20]:
# store the efficient SMM estimates
delta_hat2 = efficient_SMM_result2.x[0]
mu_hat2 = efficient_SMM_result2.x[1]
sigma_hat2 = efficient_SMM_result2.x[2]

# calculate the std error following Pakes and Pollard (1989)
sample_SE_delta2, sample_SE_mu2, sample_SE_sigma2 = Pakes_Pollard_SE_wrong(efficient_SMM_result2, W_hat2, 0.001, data_observable)

# store the confidence level
CI_constant = norm.ppf(0.975, 0, 1)

print("misspecified SMM delta_hat: ", delta_hat2)
print("misspecified SMM mu_hat: ", mu_hat2)
print("misspecified SMM sigma_hat: ", sigma_hat2)

print("misspecified SMM CI for delta_hat: [", delta_hat2 - CI_constant*sample_SE_delta2,",", delta_hat2 + CI_constant*sample_SE_delta2, "]")
print("misspecified SMM CI for mu_hat: [", mu_hat2 - CI_constant*sample_SE_mu2,",", mu_hat2 + CI_constant*sample_SE_mu2, "]")
print("misspecified SMM CI for sigma_hat: [", sigma_hat2 - CI_constant*sample_SE_sigma2,",", sigma_hat2 + CI_constant*sample_SE_sigma2, "]")

misspecified SMM delta_hat:  -2.895653856478084
misspecified SMM mu_hat:  3.619669582312371
misspecified SMM sigma_hat:  5.328483163931301
misspecified SMM CI for delta_hat: [ -3.2758601410415693 , -2.5154475719145983 ]
misspecified SMM CI for mu_hat: [ 2.9447359149698062 , 4.294603249654935 ]
misspecified SMM CI for sigma_hat: [ 4.529268549375564 , 6.127697778487038 ]


# Q4. SMM using market level information

In [68]:
market_list = [i for i in range(market_number)]
outcomes = []
for B in range(100):
    random.seed(B)
    selection_list = random.choices(market_list, k=250)
    
    start_time = time.time()
    bnds = ((0, 3), (0, 3), (0, 3))
    MSM_result = minimize(MSM_estimator_0204_bootstrap, [0.5, 1, 0.5], args = (data_observable, selection_list), method='Nelder-Mead', bounds = bnds, options={'maxiter':300})
    outcomes.append(MSM_result.x) 

In [94]:
q4_delta_array = np.array(outcomes)[:,0]
q4_mu_array = np.array(outcomes)[:,1]
q4_sigma_array = np.array(outcomes)[:,2]

print("Average of Minimum Distance delta estimates", q4_delta_array.mean())
print("Average of Minimum Distance mu estimates", q4_mu_array.mean())
print("Average of Minimum Distance sigma estimates", q4_sigma_array.mean())

print("Empirical Confidence Interval for delta: [", np.percentile(q4_delta_array, 2.5),",", np.percentile(q4_delta_array, 97.5),"]")
print("Empirical Confidence Interval for mu: [", np.percentile(q4_mu_array, 2.5),",", np.percentile(q4_mu_array, 97.5),"]")
print("Empirical Confidence Interval for sigma: [", np.percentile(q4_sigma_array, 2.5),",", np.percentile(q4_sigma_array, 97.5),"]")

Average of Minimum Distance delta estimates 0.6455314443252769
Average of Minimum Distance mu estimates 2.1762703765670652
Average of Minimum Distance sigma estimates 1.0198574362976252
Empirical Confidence Interval for delta: [ 0.08153605483488154 , 1.2880854339810504 ]
Empirical Confidence Interval for mu: [ 1.74934902201824 , 2.5533735272340032 ]
Empirical Confidence Interval for sigma: [ 0.4337890785033024 , 1.383087492871576 ]


# Q5. Set Identification

In [23]:
df_master = pd.read_csv("df_list_9.csv")

df_2firms = df_reshape(df_master, 2)
df_3firms = df_reshape(df_master, 3)
df_4firms = df_reshape(df_master, 4)

df_2firms = bin_number_2firms(df_2firms)
df_3firms = bin_number_3firms(df_3firms)
df_4firms = bin_number_4firms(df_4firms)

freq_2firms_dict = {}
for bin_num in df_2firms['bin_num'].unique():
    freq_2firms_dict[bin_num] = Freq_Est_2firms(df_2firms, bin_num)

freq_3firms_dict = {}
for bin_num in df_3firms['bin_num'].unique():
    freq_3firms_dict[bin_num] = Freq_Est_3firms(df_3firms, bin_num)

freq_4firms_dict = {}
for bin_num in df_4firms['bin_num'].unique():
    freq_4firms_dict[bin_num] = Freq_Est_4firms(df_4firms, bin_num)

global_opt = minimize(min_obj, [1,2,1], args = (df_2firms, df_3firms, df_4firms, freq_2firms_dict, freq_3firms_dict, freq_4firms_dict), method='Nelder-Mead', options={'maxiter':300})


In [27]:
# step 0: construct a discretized parameter space
delta_grid = np.linspace(0,3,20)
mu_grid = np.linspace(0,3,20)
sigma_grid = np.linspace(0,3,20)
parameter_space = [(a,b,c) for a in delta_grid for b in mu_grid for c in sigma_grid]

Q_n_list = [min_obj_ray.remote(parameter, df_2firms, df_3firms, df_4firms, freq_2firms_dict, freq_3firms_dict, freq_4firms_dict)for parameter in parameter_space]
Q_n_array = np.array(ray.get(Q_n_list))

In [28]:
# step 1: initialize with c_0 
df_level_set = df({"parameter_space":parameter_space,
                  "Q_n_values": Q_n_array})
df_level_set['Q_n_gap'] = df_level_set['Q_n_values'] - global_opt.fun

c_0 = 2*global_opt.fun
criterion = c_0 / market_number

In [29]:
# step 2: construct the level set C(c_0)
df_filtered = df_level_set.loc[df_level_set['Q_n_gap'] <= criterion]

In [30]:
# step 3: update c_1 from bootstrapping
B = 100
b = 80
parameter_space_c0 = np.array(df_filtered['parameter_space'])
market_id_list = [i for i in range(250)]

T = [mieq_bootstrap.remote(j, b, data_observable, parameter_space_c0, market_id_list) for j in range(B)]
T_array = np.array(ray.get(T))
c_1 = np.percentile(T_array, 5)

In [31]:
# step 4: go further to update cut off level to c_2
criterion = c_1 / market_number
df_filtered = df_level_set.loc[df_level_set['Q_n_gap'] <= criterion]
parameter_space_c1 = np.array(df_filtered['parameter_space'])
market_id_list = [i for i in range(250)]

T = [mieq_bootstrap.remote(j, b, data_observable, parameter_space_c1, market_id_list) for j in range(B)]
T_array = np.array(ray.get(T))
c_2 = np.percentile(T_array, 5)

In [34]:
# report the confidence region
criterion = c_2 / market_number
df_filtered = df_level_set.loc[df_level_set['Q_n_gap'] <= criterion]
delta_list = [candidate[0] for candidate in np.array(df_filtered['parameter_space'])]
mu_list = [candidate[1] for candidate in np.array(df_filtered['parameter_space'])]
sigma_list = [candidate[2] for candidate in np.array(df_filtered['parameter_space'])]

print("initial cut off c_0: ", c_0)
print("updated cut off c_1: ", c_1)
print("updated cut off c_2: ", c_2)

print("confidence bound for delta: [", np.array(delta_list).min(),",", np.array(delta_list).max(), "]")
print("confidence bound for mu: [", np.array(mu_list).min(),",", np.array(mu_list).max(), "]")
print("confidence bound for sigma: [", np.array(sigma_list).min(),",", np.array(sigma_list).max(), "]")

initial cut off c_0:  0.8641576826890175
updated cut off c_1:  0.871716432001008
updated cut off c_2:  0.863857911752052
confidence bound for delta: [ 0.7894736842105263 , 1.7368421052631577 ]
confidence bound for mu: [ 1.263157894736842 , 2.052631578947368 ]
confidence bound for sigma: [ 1.7368421052631577 , 2.2105263157894735 ]
